# 🛰️ Satellite Attitude Control System - Complete Guide

## Introduction

This notebook presents a complete satellite attitude control system, including:
- 3D rotation mathematics
- PID control adapted for space systems
- Realistic physics simulation
- Interactive visualization

## Learning Objectives

By the end of this tutorial, you will understand:
1. Rotation matrices and Euler angles
2. The gimbal lock problem
3. PID controllers for attitude
4. Satellite rotational dynamics
5. 3D visualization techniques

## 📚 1. Library Imports

Let's start by importing all the necessary libraries for our project.

In [ ]:
# Libraries for numerical calculations
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

# Libraries for interactive visualization
import pygame
import math
import random
from dataclasses import dataclass
from typing import Tuple, List

# Configure matplotlib for inline display
%matplotlib inline
plt.style.use('dark_background')

print("✅ All libraries imported successfully!")

## 🔧 2. 3D Rotation Mathematics

### 2.1 Rotation Matrices

Rotation matrices are fundamental for describing object orientation in 3D space.

In [ ]:
def create_rotation_matrix(roll, pitch, yaw):
    """
    Creates a 3D rotation matrix from Euler angles.
    
    Parameters:
    - roll: rotation around X-axis (degrees)
    - pitch: rotation around Y-axis (degrees) 
    - yaw: rotation around Z-axis (degrees)
    
    Returns:
    - 3x3 rotation matrix
    """
    # Convert to radians
    roll = np.radians(roll)
    pitch = np.radians(pitch)
    yaw = np.radians(yaw)
    
    # Rotation matrix around X (roll)
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    
    # Rotation matrix around Y (pitch)
    Ry = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    
    # Rotation matrix around Z (yaw)
    Rz = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    
    # Rotation composition: Rz @ Ry @ Rx
    return Rz @ Ry @ Rx

# Test the function
R = create_rotation_matrix(30, 45, 60)
print("Rotation matrix for (30°, 45°, 60°):")
print(R)
print(f"\nDeterminant (should be 1): {np.linalg.det(R):.6f}")

### 2.2 Rotation Visualization

Let's create a function to visualize rotation effects on a 3D cube.

In [ ]:
# Define the basic cube
vertices = np.array([
    [-1, -1, -1], [1, -1, -1], [1, 1, -1], [-1, 1, -1],  # Bottom face
    [-1, -1, 1], [1, -1, 1], [1, 1, 1], [-1, 1, 1]       # Top face
])

# Define cube edges
edges = [
    [0,1], [1,2], [2,3], [3,0],  # Bottom face
    [4,5], [5,6], [6,7], [7,4],  # Top face
    [0,4], [1,5], [2,6], [3,7]   # Vertical edges
]

def visualize_rotation(roll, pitch, yaw, title="3D Rotation"):
    """
    Visualizes rotation effects on a 3D cube.
    """
    # Create rotation matrix
    rotation_matrix = create_rotation_matrix(roll, pitch, yaw)
    
    # Apply rotation to vertices
    rotated_vertices = vertices @ rotation_matrix.T
    
    # Create 3D figure
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Draw the cube
    for edge in edges:
        points = rotated_vertices[edge]
        ax.plot([points[0, 0], points[1, 0]],
                [points[0, 1], points[1, 1]],
                [points[0, 2], points[1, 2]], 
                'cyan', linewidth=2)
    
    # Draw reference axes
    ax.plot([0, 2], [0, 0], [0, 0], 'r-', linewidth=3, label='X (Roll)')
    ax.plot([0, 0], [0, 2], [0, 0], 'g-', linewidth=3, label='Y (Pitch)')
    ax.plot([0, 0], [0, 0], [0, 2], 'b-', linewidth=3, label='Z (Yaw)')
    
    # Draw satellite axes
    sat_x = np.array([1.5, 0, 0]) @ rotation_matrix.T
    sat_y = np.array([0, 1.5, 0]) @ rotation_matrix.T
    sat_z = np.array([0, 0, 1.5]) @ rotation_matrix.T
    
    ax.plot([0, sat_x[0]], [0, sat_x[1]], [0, sat_x[2]], 'r--', linewidth=2, alpha=0.7)
    ax.plot([0, sat_y[0]], [0, sat_y[1]], [0, sat_y[2]], 'g--', linewidth=2, alpha=0.7)
    ax.plot([0, sat_z[0]], [0, sat_z[1]], [0, sat_z[2]], 'b--', linewidth=2, alpha=0.7)
    
    # Configure display
    ax.set_xlim([-2.5, 2.5])
    ax.set_ylim([-2.5, 2.5])
    ax.set_zlim([-2.5, 2.5])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'{title}\nRoll={roll}°, Pitch={pitch}°, Yaw={yaw}°')
    ax.legend()
    
    plt.show()

# Visualization examples
print("🎯 Rotation visualizations:")
visualize_rotation(0, 0, 0, "Reference position")
visualize_rotation(45, 0, 0, "Roll rotation only")
visualize_rotation(0, 45, 0, "Pitch rotation only")
visualize_rotation(0, 0, 45, "Yaw rotation only")

## ⚠️ 3. The Gimbal Lock Problem

Gimbal lock is a critical phenomenon in attitude control that occurs when pitch reaches ±90°.

In [ ]:
def analyze_gimbal_lock(pitch_angle):
    """
    Analyzes gimbal lock effects for different pitch angles.
    """
    print(f"\n📊 Gimbal Lock Analysis for Pitch = {pitch_angle}°")
    print("=" * 50)
    
    # Test rotation matrices
    base_matrix = create_rotation_matrix(30, pitch_angle, 30)
    roll_matrix = create_rotation_matrix(31, pitch_angle, 30)  # +1° in roll
    yaw_matrix = create_rotation_matrix(30, pitch_angle, 31)   # +1° in yaw
    
    # Calculate differences
    roll_diff = roll_matrix @ np.linalg.inv(base_matrix)
    yaw_diff = yaw_matrix @ np.linalg.inv(base_matrix)
    
    # Measure similarity
    similarity = np.trace(roll_diff @ yaw_diff.T) / 3.0
    
    print(f"Similarity between roll and yaw: {similarity:.6f}")
    print(f"Gimbal lock risk: {abs(abs(similarity) - 1.0):.6f}")
    print("(Closer to 0 = higher risk)")
    
    if abs(abs(similarity) - 1.0) < 0.01:
        print("🚨 WARNING: Gimbal lock detected!")
    elif abs(abs(similarity) - 1.0) < 0.1:
        print("⚠️  Close to gimbal lock")
    else:
        print("✅ Normal rotation")
    
    return similarity

# Gimbal lock test
test_angles = [0, 30, 60, 85, 89, 90, 91, 95]
similarities = []

for angle in test_angles:
    sim = analyze_gimbal_lock(angle)
    similarities.append(abs(abs(sim) - 1.0))

# Gimbal lock risk graph
plt.figure(figsize=(12, 6))
plt.plot(test_angles, similarities, 'ro-', linewidth=2, markersize=8)
plt.axhline(y=0.01, color='red', linestyle='--', alpha=0.7, label='Critical threshold')
plt.xlabel('Pitch Angle (degrees)')
plt.ylabel('Gimbal Lock Risk')
plt.title('Gimbal Lock Risk Analysis')
plt.grid(True, alpha=0.3)
plt.legend()
plt.yscale('log')
plt.show()

print("\n📈 The graph shows maximum risk near ±90° pitch.")

## 🎯 4. Orientation Error Calculation

To control attitude, we must calculate the error between current and desired orientation.

In [ ]:
def calculate_orientation_error(current_roll, current_pitch, current_yaw,
                                desired_roll, desired_pitch, desired_yaw):
    """
    Calculates orientation error in axis-angle representation.
    
    This method avoids Euler angle problems and provides
    the most direct rotation to correct the error.
    """
    # Rotation matrices
    R_current = create_rotation_matrix(current_roll, current_pitch, current_yaw)
    R_desired = create_rotation_matrix(desired_roll, desired_pitch, desired_yaw)
    
    # Error matrix
    R_error = R_desired @ R_current.T
    
    # Extract axis and angle
    trace = np.trace(R_error)
    angle = np.arccos(np.clip((trace - 1) / 2, -1, 1))
    
    if angle > 1e-6:
        axis_x = (R_error[2, 1] - R_error[1, 2]) / (2 * np.sin(angle))
        axis_y = (R_error[0, 2] - R_error[2, 0]) / (2 * np.sin(angle))
        axis_z = (R_error[1, 0] - R_error[0, 1]) / (2 * np.sin(angle))
        
        # Normalization
        axis_magnitude = np.sqrt(axis_x**2 + axis_y**2 + axis_z**2)
        if axis_magnitude > 1e-6:
            axis_x /= axis_magnitude
            axis_y /= axis_magnitude
            axis_z /= axis_magnitude
        else:
            axis_x, axis_y, axis_z = 0, 0, 1
            angle = 0
    else:
        axis_x, axis_y, axis_z = 0, 0, 1
        angle = 0
    
    angle_degrees = np.degrees(angle)
    return axis_x, axis_y, axis_z, angle_degrees

# Error function tests
print("🧪 Orientation error calculation tests:")
print("=" * 50)

# Test 1: No error
axis_x, axis_y, axis_z, angle = calculate_orientation_error(0, 0, 0, 0, 0, 0)
print(f"Test 1 - No error: Angle = {angle:.3f}°")

# Test 2: Simple rotation
axis_x, axis_y, axis_z, angle = calculate_orientation_error(0, 0, 0, 30, 0, 0)
print(f"Test 2 - Roll 30°: Axis=({axis_x:.3f}, {axis_y:.3f}, {axis_z:.3f}), Angle={angle:.3f}°")

# Test 3: Complex rotation
axis_x, axis_y, axis_z, angle = calculate_orientation_error(0, 0, 0, 30, 45, 60)
print(f"Test 3 - Complex rotation: Axis=({axis_x:.3f}, {axis_y:.3f}, {axis_z:.3f}), Angle={angle:.3f}°")

# Error visualization
def visualize_error(current_angles, desired_angles):
    """Visualizes orientation error."""
    axis_x, axis_y, axis_z, angle = calculate_orientation_error(*current_angles, *desired_angles)
    
    fig = plt.figure(figsize=(12, 5))
    
    # Current orientation
    ax1 = fig.add_subplot(131, projection='3d')
    R_current = create_rotation_matrix(*current_angles)
    rotated_vertices = vertices @ R_current.T
    
    for edge in edges:
        points = rotated_vertices[edge]
        ax1.plot([points[0, 0], points[1, 0]],
                 [points[0, 1], points[1, 1]],
                 [points[0, 2], points[1, 2]], 'red', linewidth=2)
    
    ax1.set_title('Current Orientation')
    ax1.set_xlim([-2, 2]); ax1.set_ylim([-2, 2]); ax1.set_zlim([-2, 2])
    
    # Desired orientation
    ax2 = fig.add_subplot(132, projection='3d')
    R_desired = create_rotation_matrix(*desired_angles)
    rotated_vertices = vertices @ R_desired.T
    
    for edge in edges:
        points = rotated_vertices[edge]
        ax2.plot([points[0, 0], points[1, 0]],
                 [points[0, 1], points[1, 1]],
                 [points[0, 2], points[1, 2]], 'green', linewidth=2)
    
    ax2.set_title('Desired Orientation')
    ax2.set_xlim([-2, 2]); ax2.set_ylim([-2, 2]); ax2.set_zlim([-2, 2])
    
    # Error axis
    ax3 = fig.add_subplot(133, projection='3d')
    ax3.quiver(0, 0, 0, axis_x, axis_y, axis_z, color='orange', arrow_length_ratio=0.1, linewidth=3)
    ax3.text(axis_x, axis_y, axis_z, f'Error: {angle:.1f}°', fontsize=12)
    
    ax3.set_title(f'Correction Axis\nAngle: {angle:.2f}°')
    ax3.set_xlim([-1, 1]); ax3.set_ylim([-1, 1]); ax3.set_zlim([-1, 1])
    
    plt.tight_layout()
    plt.show()

# Error visualization example
visualize_error((45, 30, -60), (0, 0, 0))

## 🎛️ 5. PID Controller for Attitude

The PID controller is the heart of our attitude control system.

In [ ]:
class AttitudeController:
    """
    PID controller specialized for 3D attitude control.
    
    The controller uses axis-angle representation to avoid
    singularities and provide robust control.
    """
    
    def __init__(self, kp=1.0, ki=0.1, kd=0.05):
        """
        Initialize PID controller.
        
        Parameters:
        - kp: Proportional gain (immediate response to error)
        - ki: Integral gain (correction for persistent errors)
        - kd: Derivative gain (damping and stability)
        """
        self.kp = kp
        self.ki = ki
        self.kd = kd
        
        # Controller internal state
        self.integral_error = np.array([0.0, 0.0, 0.0])
        self.previous_error_vector = None
        self.dt = 0.1  # Time step
    
    def update(self, current_roll, current_pitch, current_yaw,
               desired_roll, desired_pitch, desired_yaw):
        """
        Calculate PID control command.
        
        Returns:
        - (torque_x, torque_y, torque_z): Control torques
        """
        # Error calculation
        axis_x, axis_y, axis_z, error_angle = calculate_orientation_error(
            current_roll, current_pitch, current_yaw,
            desired_roll, desired_pitch, desired_yaw
        )
        
        # 3D error vector
        error_vector = np.array([axis_x, axis_y, axis_z]) * np.radians(error_angle)
        
        # Proportional term
        proportional_term = self.kp * error_vector
        
        # Integral term
        self.integral_error += error_vector * self.dt
        integral_term = self.ki * self.integral_error
        
        # Derivative term
        if self.previous_error_vector is not None:
            error_rate = (error_vector - self.previous_error_vector) / self.dt
            derivative_term = self.kd * error_rate
        else:
            derivative_term = np.array([0.0, 0.0, 0.0])
        
        # Total command
        control_output = proportional_term + integral_term + derivative_term
        
        # Save for next iteration
        self.previous_error_vector = error_vector.copy()
        
        return control_output[0], control_output[1], control_output[2]
    
    def reset(self):
        """Reset controller internal state."""
        self.integral_error = np.array([0.0, 0.0, 0.0])
        self.previous_error_vector = None

# Controller test
print("🎛️ PID controller test:")
print("=" * 40)

controller = AttitudeController(kp=2.0, ki=0.5, kd=0.1)

# Correction simulation
current_orientation = (30, 45, 60)
desired_orientation = (0, 0, 0)

print(f"Current orientation: {current_orientation}")
print(f"Desired orientation: {desired_orientation}")

torque_x, torque_y, torque_z = controller.update(*current_orientation, *desired_orientation)
torque_magnitude = np.sqrt(torque_x**2 + torque_y**2 + torque_z**2)

print(f"\nPID command:")
print(f"Torque X: {torque_x:.4f}")
print(f"Torque Y: {torque_y:.4f}")
print(f"Torque Z: {torque_z:.4f}")
print(f"Total magnitude: {torque_magnitude:.4f}")

## 🚀 6. Satellite Physics Simulation

Let's create a realistic physics simulation including inertia and damping.

In [ ]:
class SatelliteSimulation:
    """
    Complete physics simulation of a satellite with attitude control.
    """
    
    def __init__(self, initial_roll=0, initial_pitch=0, initial_yaw=0):
        # Satellite state
        self.roll = initial_roll
        self.pitch = initial_pitch
        self.yaw = initial_yaw
        
        # Angular velocities
        self.roll_rate = 0.0
        self.pitch_rate = 0.0
        self.yaw_rate = 0.0
        
        # Physical properties
        self.inertia = 1.0      # Moment of inertia
        self.damping = 0.05     # Damping
        self.dt = 0.1           # Time step
        
        # Targets
        self.target_roll = 0.0
        self.target_pitch = 0.0
        self.target_yaw = 0.0
        
        # Controller
        self.controller = AttitudeController(kp=3.0, ki=0.2, kd=0.8)
        
        # History for plots
        self.time_history = []
        self.error_history = []
        self.control_history = []
        self.roll_history = []
        self.pitch_history = []
        self.yaw_history = []
        self.time = 0.0
    
    def apply_physics_update(self, torque_x, torque_y, torque_z):
        """
        Update satellite physics.
        
        Uses Newton's equations for rotation:
        Torque = Inertia × Angular_Acceleration
        """
        # Angular accelerations
        roll_accel = torque_x / self.inertia
        pitch_accel = torque_y / self.inertia
        yaw_accel = torque_z / self.inertia
        
        # Update velocities (with damping)
        self.roll_rate = self.roll_rate * (1 - self.damping) + roll_accel * self.dt
        self.pitch_rate = self.pitch_rate * (1 - self.damping) + pitch_accel * self.dt
        self.yaw_rate = self.yaw_rate * (1 - self.damping) + yaw_accel * self.dt
        
        # Update positions
        self.roll += self.roll_rate * self.dt
        self.pitch += self.pitch_rate * self.dt
        self.yaw += self.yaw_rate * self.dt
        
        # Normalize angles
        self.roll = ((self.roll + 180) % 360) - 180
        self.pitch = ((self.pitch + 180) % 360) - 180
        self.yaw = ((self.yaw + 180) % 360) - 180
    
    def update_simulation(self):
        """
        Complete simulation step: Sense → Think → Act
        """
        # 1. SENSE: Error calculation
        axis_x, axis_y, axis_z, error_angle = calculate_orientation_error(
            self.roll, self.pitch, self.yaw,
            self.target_roll, self.target_pitch, self.target_yaw
        )
        
        # 2. THINK: Command calculation
        torque_x, torque_y, torque_z = self.controller.update(
            self.roll, self.pitch, self.yaw,
            self.target_roll, self.target_pitch, self.target_yaw
        )
        
        # 3. ACT: Physics application
        self.apply_physics_update(torque_x, torque_y, torque_z)
        
        # 4. RECORD
        self.time += self.dt
        self.time_history.append(self.time)
        self.error_history.append(error_angle)
        self.control_history.append(np.sqrt(torque_x**2 + torque_y**2 + torque_z**2))
        self.roll_history.append(self.roll)
        self.pitch_history.append(self.pitch)
        self.yaw_history.append(self.yaw)
        
        return self.roll, self.pitch, self.yaw, error_angle
    
    def apply_disturbance(self, strength=20.0):
        """Apply random disturbance."""
        self.roll += np.random.uniform(-strength, strength)
        self.pitch += np.random.uniform(-strength, strength)
        self.yaw += np.random.uniform(-strength, strength)

# Simulation test
print("🚀 Physics simulation test:")
print("=" * 40)

# Create simulation with initial error
sim = SatelliteSimulation(initial_roll=45, initial_pitch=30, initial_yaw=-60)

print(f"Initial orientation: ({sim.roll:.1f}°, {sim.pitch:.1f}°, {sim.yaw:.1f}°)")
print(f"Target orientation: ({sim.target_roll:.1f}°, {sim.target_pitch:.1f}°, {sim.target_yaw:.1f}°)")

# Simulate a few steps
print("\nSimulation evolution:")
for i in range(5):
    roll, pitch, yaw, error = sim.update_simulation()
    print(f"t={sim.time:.1f}s: ({roll:.1f}°, {pitch:.1f}°, {yaw:.1f}°) - Error: {error:.2f}°")

## 📊 7. Performance Analysis

Let's analyze controller performance with different settings.

In [ ]:
def compare_controllers():
    """
    Compare performance of different PID settings.
    """
    # Configurations to test
    configs = [
        {"name": "Conservative", "kp": 1.0, "ki": 0.1, "kd": 0.2, "color": "blue"},
        {"name": "Aggressive", "kp": 5.0, "ki": 0.5, "kd": 1.0, "color": "red"},
        {"name": "Optimal", "kp": 3.0, "ki": 0.2, "kd": 0.8, "color": "green"},
    ]
    
    plt.figure(figsize=(15, 10))
    
    for i, config in enumerate(configs):
        # Create new simulation
        sim = SatelliteSimulation(initial_roll=45, initial_pitch=30, initial_yaw=-60)
        sim.controller = AttitudeController(kp=config["kp"], ki=config["ki"], kd=config["kd"])
        
        # Run simulation
        for _ in range(100):
            sim.update_simulation()
            
            # Add disturbance midway
            if len(sim.time_history) == 50:
                sim.apply_disturbance(10.0)
        
        # Plots
        plt.subplot(2, 3, 1)
        plt.plot(sim.time_history, sim.error_history, color=config["color"], 
                 label=config["name"], linewidth=2)
        plt.title("Orientation Error")
        plt.xlabel("Time (s)")
        plt.ylabel("Error (°)")
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        plt.subplot(2, 3, 2)
        plt.plot(sim.time_history, sim.control_history, color=config["color"], 
                 label=config["name"], linewidth=2)
        plt.title("Control Effort")
        plt.xlabel("Time (s)")
        plt.ylabel("Magnitude")
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        plt.subplot(2, 3, 3)
        plt.plot(sim.time_history, sim.roll_history, color=config["color"], 
                 label=config["name"], linewidth=2)
        plt.title("Roll Evolution")
        plt.xlabel("Time (s)")
        plt.ylabel("Angle (°)")
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        plt.subplot(2, 3, 4)
        plt.plot(sim.time_history, sim.pitch_history, color=config["color"], 
                 label=config["name"], linewidth=2)
        plt.title("Pitch Evolution")
        plt.xlabel("Time (s)")
        plt.ylabel("Angle (°)")
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        plt.subplot(2, 3, 5)
        plt.plot(sim.time_history, sim.yaw_history, color=config["color"], 
                 label=config["name"], linewidth=2)
        plt.title("Yaw Evolution")
        plt.xlabel("Time (s)")
        plt.ylabel("Angle (°)")
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # Calculate metrics
        settling_time = next((t for t, e in zip(sim.time_history, sim.error_history) if e < 1.0), None)
        overshoot = max(sim.control_history) if sim.control_history else 0
        steady_state_error = np.mean(sim.error_history[-10:]) if len(sim.error_history) > 10 else 0
        
        print(f"\n📊 Metrics for {config['name']}:")
        print(f"   Settling time: {settling_time:.1f}s" if settling_time else "   Not settled")
        print(f"   Max overshoot: {overshoot:.2f}")
        print(f"   Final error: {steady_state_error:.3f}°")
    
    # Comparison table
    plt.subplot(2, 3, 6)
    plt.axis('off')
    table_data = []
    for config in configs:
        table_data.append([config['name'], f"kp={config['kp']}", f"ki={config['ki']}", f"kd={config['kd']}"])
    
    table = plt.table(cellText=table_data,
                      colLabels=['Configuration', 'kp', 'ki', 'kd'],
                      cellLoc='center',
                      loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    plt.title("PID Parameters")
    
    plt.tight_layout()
    plt.show()

# Run comparison
print("🔬 PID controller comparison:")
compare_controllers()

## 🎮 8. Complete Interactive Demonstration

Let's create a complete demonstration that combines all elements.

In [ ]:
def create_interactive_demo():
    """
    Creates an interactive demonstration of the attitude control system.
    """
    # Initial configuration
    sim = SatelliteSimulation(initial_roll=45, initial_pitch=30, initial_yaw=-60)
    
    # Figure configuration
    fig = plt.figure(figsize=(16, 12))
    
    # Subplot layouts
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # Main 3D view
    ax_3d = fig.add_subplot(gs[0:2, 0:2], projection='3d')
    ax_3d.set_title('Real-time Satellite', fontsize=14, fontweight='bold')
    
    # Performance plots
    ax_error = fig.add_subplot(gs[0, 2])
    ax_error.set_title('Orientation Error')
    ax_error.set_ylabel('Error (°)')
    ax_error.grid(True, alpha=0.3)
    
    ax_control = fig.add_subplot(gs[0, 3])
    ax_control.set_title('Control Effort')
    ax_control.set_ylabel('Magnitude')
    ax_control.grid(True, alpha=0.3)
    
    ax_angles = fig.add_subplot(gs[1, 2:4])
    ax_angles.set_title('Angle Evolution')
    ax_angles.set_ylabel('Angles (°)')
    ax_angles.grid(True, alpha=0.3)
    
    # Information panel
    ax_info = fig.add_subplot(gs[2, :])
    ax_info.axis('off')
    
    def animate(frame):
        """Animation function called each frame."""
        
        # Add periodic disturbances
        if frame > 0 and frame % 100 == 0:
            sim.apply_disturbance(15.0)
            print(f"⚡ Disturbance applied at t={sim.time:.1f}s")
        
        # Change target periodically
        if frame > 0 and frame % 150 == 0:
            sim.target_roll = np.random.uniform(-30, 30)
            sim.target_pitch = np.random.uniform(-30, 30)
            sim.target_yaw = np.random.uniform(-30, 30)
            print(f"🎯 New target: ({sim.target_roll:.1f}°, {sim.target_pitch:.1f}°, {sim.target_yaw:.1f}°)")
        
        # Update simulation
        roll, pitch, yaw, error = sim.update_simulation()
        
        # Update 3D view
        ax_3d.clear()
        ax_3d.set_title(f'Satellite - t={sim.time:.1f}s', fontweight='bold')
        
        # Draw satellite
        rotation_matrix = create_rotation_matrix(roll, pitch, yaw)
        rotated_vertices = vertices @ rotation_matrix.T
        
        # Satellite cube
        for edge in edges:
            points = rotated_vertices[edge]
            ax_3d.plot([points[0, 0], points[1, 0]],
                       [points[0, 1], points[1, 1]],
                       [points[0, 2], points[1, 2]], 
                       'cyan', linewidth=2, alpha=0.8)
        
        # Reference axes
        ax_3d.plot([0, 2], [0, 0], [0, 0], 'r-', linewidth=3, alpha=0.7)
        ax_3d.plot([0, 0], [0, 2], [0, 0], 'g-', linewidth=3, alpha=0.7)
        ax_3d.plot([0, 0], [0, 0], [0, 2], 'b-', linewidth=3, alpha=0.7)
        
        # Satellite axes
        sat_x = np.array([1.5, 0, 0]) @ rotation_matrix.T
        sat_y = np.array([0, 1.5, 0]) @ rotation_matrix.T
        sat_z = np.array([0, 0, 1.5]) @ rotation_matrix.T
        
        ax_3d.plot([0, sat_x[0]], [0, sat_x[1]], [0, sat_x[2]], 'r--', linewidth=2, alpha=0.9)
        ax_3d.plot([0, sat_y[0]], [0, sat_y[1]], [0, sat_y[2]], 'g--', linewidth=2, alpha=0.9)
        ax_3d.plot([0, sat_z[0]], [0, sat_z[1]], [0, sat_z[2]], 'b--', linewidth=2, alpha=0.9)
        
        ax_3d.set_xlim([-2.5, 2.5])
        ax_3d.set_ylim([-2.5, 2.5])
        ax_3d.set_zlim([-2.5, 2.5])
        
        # Update performance plots
        if len(sim.time_history) > 1:
            # Error
            ax_error.clear()
            ax_error.plot(sim.time_history, sim.error_history, 'r-', linewidth=2)
            ax_error.set_title('Orientation Error')
            ax_error.set_ylabel('Error (°)')
            ax_error.grid(True, alpha=0.3)
            ax_error.set_xlim(max(0, sim.time - 20), sim.time + 2)
            
            # Control
            ax_control.clear()
            ax_control.plot(sim.time_history, sim.control_history, 'b-', linewidth=2)
            ax_control.set_title('Control Effort')
            ax_control.set_ylabel('Magnitude')
            ax_control.grid(True, alpha=0.3)
            ax_control.set_xlim(max(0, sim.time - 20), sim.time + 2)
            
            # Angles
            ax_angles.clear()
            ax_angles.plot(sim.time_history, sim.roll_history, 'r-', linewidth=2, label='Roll')
            ax_angles.plot(sim.time_history, sim.pitch_history, 'g-', linewidth=2, label='Pitch')
            ax_angles.plot(sim.time_history, sim.yaw_history, 'b-', linewidth=2, label='Yaw')
            ax_angles.axhline(y=sim.target_roll, color='r', linestyle=':', alpha=0.7)
            ax_angles.axhline(y=sim.target_pitch, color='g', linestyle=':', alpha=0.7)
            ax_angles.axhline(y=sim.target_yaw, color='b', linestyle=':', alpha=0.7)
            ax_angles.set_title('Angle Evolution')
            ax_angles.set_ylabel('Angles (°)')
            ax_angles.set_xlabel('Time (s)')
            ax_angles.legend()
            ax_angles.grid(True, alpha=0.3)
            ax_angles.set_xlim(max(0, sim.time - 20), sim.time + 2)
        
        # Information panel
        ax_info.clear()
        ax_info.axis('off')
        
        info_text = f"""
        🛰️ SATELLITE ATTITUDE CONTROL SYSTEM
        
        📊 Current State:                         🎯 Targets:                            ⚙️ PID Controller:
        Roll:    {roll:7.2f}°                     Roll:    {sim.target_roll:7.2f}°        kp = {sim.controller.kp:.1f}
        Pitch:   {pitch:7.2f}°                    Pitch:   {sim.target_pitch:7.2f}°       ki = {sim.controller.ki:.1f}
        Yaw:     {yaw:7.2f}°                      Yaw:     {sim.target_yaw:7.2f}°         kd = {sim.controller.kd:.1f}
        
        📈 Performance:                           🚀 System Status:
        Error:   {error:7.2f}°                    {'🟢 STABLE' if error < 1 else '🟡 CONVERGING' if error < 5 else '🔴 CORRECTING'}
        Time:    {sim.time:7.1f}s                 Last disturbance: {frame // 100} cycles
        """
        
        ax_info.text(0.05, 0.95, info_text, transform=ax_info.transAxes, 
                     fontsize=11, verticalalignment='top', fontfamily='monospace',
                     bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))
        
        return []
    
    # Create and launch animation
    print("🎬 Launching interactive demonstration...")
    print("\n📋 Demonstration features:")
    print("   • Real-time 3D visualization")
    print("   • Performance graphs")
    print("   • Automatic disturbances")
    print("   • Target changes")
    print("   • Complete system monitoring")
    
    anim = animation.FuncAnimation(fig, animate, frames=500, interval=100, blit=False, repeat=True)
    plt.show()
    
    return anim

# Launch demonstration
demo_animation = create_interactive_demo()

## 🎮 9. Interactive PyGame Version

For a truly interactive experience, here's the code to launch the PyGame version.

In [ ]:
# Note: This code requires PyGame and should be run in an appropriate environment
# It's included here for reference and can be copied to a separate .py file

pygame_code = '''
import pygame
import numpy as np
import math
import random

# Use the code from demos/satellite_visualization.py here
# This code creates a complete interactive visualization with:
# - 3D space environment
# - Interactive controls
# - Particle effects
# - Real-time performance graphs
# - Professional user interface

def launch_pygame_demo():
    """Launch the complete PyGame demonstration."""
    from demos.satellite_visualization import main
    main()

# To launch the PyGame demo:
# launch_pygame_demo()
'''

print("🎮 PyGame code available!")
print("\n📁 To launch the complete PyGame demonstration:")
print("   1. Make sure PyGame is installed: pip install pygame")
print("   2. Run: python demos/satellite_visualization.py")
print("\n✨ The PyGame version includes:")
print("   • Immersive 3D visualization")
print("   • Interactive keyboard controls")
print("   • Advanced visual effects")
print("   • Real-time physics simulation")
print("   • Professional user interface")

# Display the code
print("\n💻 PyGame reference code:")
print(pygame_code)

## 📚 10. Conclusion and Key Learnings

### What we learned:

1. **3D Rotation Mathematics**
   - Rotation matrices and composition
   - Axis-angle representation
   - Gimbal lock problem

2. **Adaptive PID Control**
   - Proportional, integral, derivative gains
   - Singularity avoidance
   - Tuning and optimization

3. **Realistic Physics Simulation**
   - Rotational dynamics
   - Inertia and damping
   - External disturbances

4. **Advanced Visualization**
   - Real-time 3D rendering
   - Interactive user interfaces
   - Performance monitoring

### Practical applications:

- **Communication satellites**: Maintaining Earth-pointing orientation
- **Space probes**: Precise instrument pointing
- **Space stations**: Stabilization and maneuvers
- **Drones and robotics**: General attitude control

### Improvement perspectives:

- Use quaternions to completely avoid gimbal lock
- Adaptive controllers and machine learning
- Fuel constraint considerations
- Realistic sensor integration (gyroscopes, magnetometers)

---

## 🚀 Next Steps

1. **Experiment** with different PID parameters
2. **Modify** initial conditions and disturbances
3. **Implement** new control algorithms
4. **Explore** robotics applications

**Congratulations! You now master the fundamentals of space attitude control! 🎉**

In [ ]:
# Final test code - Verification of all functionalities
print("🔍 Final test of all functionalities...")
print("=" * 60)

# Test 1: Rotation matrices
R = create_rotation_matrix(30, 45, 60)
print(f"✅ Rotation matrices: determinant = {np.linalg.det(R):.6f}")

# Test 2: Error calculation
axis_x, axis_y, axis_z, angle = calculate_orientation_error(45, 30, -60, 0, 0, 0)
print(f"✅ Error calculation: {angle:.2f}° around axis ({axis_x:.3f}, {axis_y:.3f}, {axis_z:.3f})")

# Test 3: PID controller
controller = AttitudeController(kp=3.0, ki=0.2, kd=0.8)
torque_x, torque_y, torque_z = controller.update(45, 30, -60, 0, 0, 0)
torque_magnitude = np.sqrt(torque_x**2 + torque_y**2 + torque_z**2)
print(f"✅ PID controller: torque of {torque_magnitude:.4f}")

# Test 4: Simulation
sim = SatelliteSimulation(initial_roll=45, initial_pitch=30, initial_yaw=-60)
for i in range(10):
    sim.update_simulation()
final_error = sim.error_history[-1]
print(f"✅ Simulation: error reduced to {final_error:.2f}° in {sim.time:.1f}s")

print("\n🎉 All tests passed! The system works perfectly.")
print("\n📖 This notebook demonstrates a complete satellite attitude control system")
print("   with rigorous mathematics, robust control, and")
print("   professional-level interactive visualizations.")

print("\n🚀 Ready for space! 🛰️")